In [61]:
import sys  
reload(sys)  
sys.setdefaultencoding('utf-8')

In [5]:
import pandas as pd
import json
import numpy as np

In [6]:
#!/usr/bin/env python
#-*-coding=gbk-*-
 
"""
     原始数据，用于建立模型
"""
#缩水版的courses，实际数据的格式应该为 课程名\t课程简介\t课程详情，并已去除html等干扰因素
courses = [           
            u'Writing II: Rhetorical Composing',
            u'Genetics and Society: A Course for Educators',
            u'General Game Playing',
            u'Genes and the Human Condition (From Behavior to Biotechnology)',
            u'A Brief History of Humankind',
            u'New Models of Business in Society',
            u'Analyse Numrique pour Ingnieurs',
            u'Evolution: A Course for Educators',
            u'Coding the Matrix: Linear Algebra through Computer Science Applications',
            u'The Dynamic Earth: A Course for Educators',
            u'Tiny Wings\tYou have always dreamed of flying - but your wings are tiny. Luckily the world is full of beautiful hills. Use the hills as jumps - slide down, flap your wings and fly! At least for a moment - until this annoying gravity brings you back down to earth. But the next hill is waiting for you already. Watch out for the night and fly as fast as you can. ',
            u'Angry Birds Free',
            u'没有\它很相似',
            u'没有\t它很相似',
            u'没有\t他很相似',
            u'没有\t他不很相似',
            u'没有',
            u'可以没有',
            u'也没有',
            u'有没有也不管',
            u'Angry Birds Stella',
            u'Flappy Wings - FREE\tFly into freedom!A parody of the #1 smash hit game!',
            u'没有一个',
            u'没有一个2',
           ]
 


In [29]:
#df_no_dup = pd.read_csv('../data/classifier/bosonNLP/bosonNLP-classifier-tuned.csv')
#df = pd.read_csv('/Users/nhu2000/Desktop/huizhou/data/huiwenhua/huizhou_web_contentv5.csv')
df = pd.read_csv('../data/sim/dsi_sim_v1.csv')

In [32]:
df = df.dropna(subset =['body'])

In [34]:
docs = df['body'].str.split()

In [35]:
courses = docs

In [36]:
courses_name = courses

In [37]:
"""
    预处理(easy_install nltk)
"""
def pre_process_cn(courses, low_freq_filter = True):
    """
     简化的 中文+英文 预处理
        1.去掉停用词
        2.去掉标点符号
        3.处理为词干
        4.去掉低频词
 
    """
    import nltk
    import jieba.analyse
    from nltk.tokenize import word_tokenize
    
    texts_tokenized = []
    for document in courses:
        texts_tokenized_tmp = []
        for word in word_tokenize(document):
            texts_tokenized_tmp += jieba.analyse.extract_tags(word,10)
        texts_tokenized.append(texts_tokenized_tmp)   
    
    texts_filtered_stopwords = texts_tokenized
 
    #去除标点符号
    english_punctuations = [',', '.', ':', ';', '?', '(', ')', '[', ']', '&', '!', '*', '@', '#', '$', '%']
    texts_filtered = [[word for word in document if not word in english_punctuations] for document in texts_filtered_stopwords]
 
    #词干化
    from nltk.stem.lancaster import LancasterStemmer
    st = LancasterStemmer()
    texts_stemmed = [[st.stem(word) for word in docment] for docment in texts_filtered]
    
    #去除过低频词
    if low_freq_filter:
        all_stems = sum(texts_stemmed, [])
        stems_once = set(stem for stem in set(all_stems) if all_stems.count(stem) == 1)
        texts = [[stem for stem in text if stem not in stems_once] for text in texts_stemmed]
    else:
        texts = texts_stemmed
    return texts

In [39]:
lib_texts = pre_process_cn(courses)

TypeError: expected string or buffer

In [42]:
 
"""
    引入gensim，正式开始处理(easy_install gensim)
"""
 
def train_by_lsi(lib_texts):
    """
        通过LSI模型的训练
    """
    from gensim import corpora, models, similarities
 
    #为了能看到过程日志
    #import logging
    #logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
 
    dictionary = corpora.Dictionary(lib_texts)
    corpus = [dictionary.doc2bow(text) for text in lib_texts]     #doc2bow(): 将collection words 转为词袋，用两元组(word_id, word_frequency)表示
    tfidf = models.TfidfModel(corpus)
    corpus_tfidf = tfidf[corpus]
 
    #拍脑袋的：训练topic数量为10的LSI模型
    lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=50)
    index = similarities.MatrixSimilarity(lsi[corpus])     # index 是 gensim.similarities.docsim.MatrixSimilarity 实例
    
    return (index, dictionary, lsi)


In [43]:
 
    
#库建立完成 -- 这部分可能数据很大，可以预先处理好，存储起来
#(index,dictionary,lsi) = train_by_lsi(lib_texts)
(index,dictionary,lsi) = train_by_lsi(docs)


In [21]:
print index

MatrixSimilarity<87458 docs, 20 features>


In [121]:
print dictionary

Dictionary(47547 unique tokens: [u'\u5e73\u5b9a', u'\u4e94\u7ebf', u'\u7b2c\u4e8c', u'\u540e\u62a5', u'\u6d2a\u6b66']...)


In [22]:
    
    
#要处理的对象登场
target_courses = docs[501] #[u'徽州']
target_text = pre_process_cn(target_courses, low_freq_filter=False)
 


NameError: name 'pre_process_cn' is not defined

In [48]:
 
"""
对具体对象相似度匹配
"""
 
#选择一个基准数据
ml_course = df.iloc[3703]['body'].split()
ml_course = {'machine', 'learning', 'jobs'}
 
#词袋处理
ml_bow = dictionary.doc2bow(ml_course)  
 
#在上面选择的模型数据 lsi 中，计算其他数据与其的相似度
ml_lsi = lsi[ml_bow]     #ml_lsi 形式如 (topic_id, topic_value)
sims = index[ml_lsi]     #sims 是最终结果了， index[xxx] 调用内置方法 __getitem__() 来计算ml_lsi
 
#排序，为输出方便
sort_sims = sorted(enumerate(sims), key=lambda item: -item[1])

tot = 20

labels, sim_values = zip(*sort_sims[0:tot])

print 'reference doc ---', labels[0], df.iloc[labels[0]]['title']

for i in range(0, tot):
    print '------------------'
    print labels[i], sim_values[i], df.iloc[labels[i]]['title']
    print labels[i], df.iloc[labels[i]]['body']
    




#查看结果
#print sort_sims[0:10]   #看下前10个最相似的，第一个是基准数据自身
#print courses_name[sort_sims[1][0]]   #看下实际最相似的数据叫什么
#print courses_name[sort_sims[2][0]]   #看下实际最相似的数据叫什么
#print courses_name[sort_sims[3][0]]   #看下实际最相似的数据叫什么



reference doc --- 26075 scikit-learn: machine learning in Python — scikit-learn 0.16.1 documentation
------------------
26075 0.90534 scikit-learn: machine learning in Python — scikit-learn 0.16.1 documentation
26075 scikit-learn: machine learning in Python — scikit-learn 0.16.1 documentation
------------------
31755 0.788141 Machine Learning with Python at PyConES 2015 | The Official Blog of BigML.com
31755 Machine Learning with Python at PyConES 2015 | The Official Blog of BigML.com Skip to content Follow: RSS Twitter The Official Blog of BigML.com Machine Learning Made Simple SIGNUP FREE BLOG DEVELOPERS GALLERY FEATURES HOW IT WORKS BigML Tags PyConES Machine Learning with Python at PyConES 2015 by mariajesusbigml on November 30, 2015 Since we established our new European Headquarters in Valencia last July, we have been carrying the Machine Learning banner at all major tech events in the city.  This time we were proud sponsors of the third edition of  PyConES , the marquee Python ev

In [52]:
print type(index)

<class 'gensim.similarities.docsim.MatrixSimilarity'>


In [107]:
print index

MatrixSimilarity<4979 docs, 20 features>


In [122]:
ret = lsi.show_topics(20, 20)

In [123]:
import re
match = '\"(.*?)\"'
for i in range(0, 20):
    t = ret[i][1]
    s = re.findall('"([^"]*)"', t)
    #print t
    print i, ' '.join(s)


0 文化 吸附 身处 手艺人 手艺 道理 感受 六年 清光绪 嘉定 自宋 文武 徽州 状元 十年 休宁 徽派 保护 徽文化 建筑
1 文化 道理 六年 吸附 手艺 身处 清光绪 嘉定 文武 手艺人 自宋 十年 感受 状元 保护 徽州 徽文化 徽派 黄山 建筑
2 建筑 徽派 古建筑 民居 研究 雕刻 文化 古民居 古建 木雕 徽学 艺术 古村落 砖雕 保护 学术 装饰 徽商 新安 西递
3 保护 徽派 徽商 新安 旅游 古村落 生态 古民居 黄山 利用 资料 艺术 黄山市 网上 网上家园 古建筑 本站 若有 管理员 侵权
4 文化 研究 保护 徽学 歙县 本站 若有 管理员 侵权 网上 来源于 网上家园 资料 网络 徽商 绩溪 休宁 生态 联系 黟县
5 徽学 研究 建筑 若有 管理员 本站 侵权 来源于 明清 网络 网上 网上家园 社会 徽商 联系 文书 安徽 未经许可 资料 随意
6 徽商 协会会员 桐城人 年生 报业 甘臻 研究生 现为 研究 徽学 安徽省 新安 木雕 大会 雕刻 七章 第七章 经营 旅游 江淮
7 徽商 协会会员 桐城人 年生 甘臻 报业 研究生 现为 木雕 黟县 第七章 江淮 西递 新安 安徽省 七章 第七 艺术 雕刻 明清
8 徽商 休宁 新安 徽文化 休宁县 木雕 婺源 雕刻 黟县 故园 传承 屯溪 大会 文化 古村落 徽剧 医学 西递 歙县 祁门
9 新安 绩溪 医学 画派 故园 木雕 胡适 徽商 活动 徽州 民俗 雕刻 周年 汪华 汪公 休宁 学术 保护 汪王 绩溪县
10 绩溪 休宁 故园 黄山 祁门 万安 保护 休宁县 徽墨 旅游 生态 徽派 徽文化 公告 文化遗产 制作 罗盘 国家级 绩溪县 大会
11 绩溪 徽菜 徽剧 黄山 休宁 木雕 休宁县 屯溪 雕刻 黄山市 歙县 万安 新安 徽文化 老街 古村落 罗盘 胡适 民俗 徽班
12 黟县 屯溪 老街 西递 旅游 祁门 宏村 建筑 胡适 徽商 绩溪 故园 保护 木雕 古建筑 学术 周年 祁门县 徽剧 屯溪区
13 徽剧 绩溪 徽班 徽菜 徽文化 京剧 徽派 旅游 歙县 故园 黄山 休宁 文化遗产 研究 胡适 四喜 西递 建筑 戏曲 进京
14 徽剧 新安 保护 徽商 徽班 中国 徽学 故园 生态 屯溪 画派 历史 老街 黄山 文书 研究 医学 京剧 休宁县 徽菜
15 歙县 